In [ ]:
import cv2
import numpy as np

# Load video
video_path = "combscroll.mp4"
cap = cv2.VideoCapture(video_path)

# Get frame rate and total frames
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Original FPS
frame_interval = fps  # Process one frame per second
frame_count = 0
scroll_timestamps = []
scrolling = False  # Flag to track scrolling start and end

# Read the first frame
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY) if ret else None

while cap.isOpened():
    # Set video position to extract frame at 1-second intervals
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count)

    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compute Optical Flow (Farneback method)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prev_gray = gray  # Update previous frame

    # Extract vertical movement component
    vertical_movement = flow[..., 1]  # Y-axis movement

    # Compute mean absolute vertical movement
    avg_vertical_flow = np.mean(np.abs(vertical_movement))

    # Define scrolling threshold (tune this based on your video)
    SCROLL_THRESHOLD = 1.5  # Adjust for sensitivity

    # Get the current timestamp in seconds
    current_time = frame_count / fps

    if avg_vertical_flow > SCROLL_THRESHOLD:
        if not scrolling:
            scroll_timestamps.append((current_time, None))
            scrolling = True
    else:
        if scrolling:
            scroll_timestamps[-1] = (scroll_timestamps[-1][0], current_time)
            scrolling = False

    # Move to the next second (skip frames for efficiency)
    frame_count += frame_interval

cap.release()
cv2.destroyAllWindows()

# Print scrolling timestamps
for start, end in scroll_timestamps:
    print(f"Scroll detected from {start:.2f}s to {end:.2f}s")


In [ ]:
# maybe try to use only the center part of the video?
# .

In [6]:
import os
import cv2
import numpy as np

video_path = "combscroll.mp4"
cap = cv2.VideoCapture(video_path)
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv2.add(frame, mask)
    cv2.imshow('frame', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
cv2.destroyAllWindows()

No frames grabbed!


In [8]:
import cv2
import numpy as np

# Load video
video_path = "combscroll.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_path = "modified_videos"

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Define center crop region (e.g., middle 50% of width and height)
crop_x_start = frame_width // 4
crop_x_end = 3 * frame_width // 4
crop_y_start = frame_height // 4
crop_y_end = 3 * frame_height // 4

# Params for ShiTomasi corner detection
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Parameters for lucas kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in the center region
ret, old_frame = cap.read()
if not ret:
    raise ValueError("Failed to read video.")

# Convert to grayscale
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Crop to center
old_gray_center = old_gray[crop_y_start:crop_y_end, crop_x_start:crop_x_end]

# Detect features in the cropped region
p0 = cv2.goodFeaturesToTrack(old_gray_center, mask=None, **feature_params)

# Adjust coordinates to match original frame
if p0 is not None:
    p0[:, 0, 0] += crop_x_start
    p0[:, 0, 1] += crop_y_start

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray_center = frame_gray[crop_y_start:crop_y_end, crop_x_start:crop_x_end]

    # Calculate optical flow in the center region
    if p0 is not None:
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            # Draw the tracks only in the center region
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                if crop_x_start <= a <= crop_x_end and crop_y_start <= b <= crop_y_end:
                    mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
                    frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

            p0 = good_new.reshape(-1, 1, 2)

    # Overlay the mask on the frame
    img = cv2.add(frame, mask)

    # Write the frame to output video
    out.write(img)

    # Update previous frame
    old_gray = frame_gray.copy()

# Release everything
cap.release()
out.release()

# Provide the download link
output_path


'modified_videos'

In [18]:
import cv2

# Set file paths
input_video_path = "combscroll.mp4"
output_video_path = "C:/Users/supia/Documents/GitHub/Accessibility-AI-and-Video/modified_videos/cropped_video.mp4"

# Open the video file
cap = cv2.VideoCapture(input_video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# Define cropping region (center 50% of width and height)
crop_x_start = frame_width // 4
crop_x_end = 3 * frame_width // 4
crop_y_start = frame_height // 4
crop_y_end = 3 * frame_height // 4
cropped_width = crop_x_end - crop_x_start
cropped_height = crop_y_end - crop_y_start

# Create a VideoWriter object
out = cv2.VideoWriter(output_video_path, fps, fourcc, (640, 480))

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break  # End of video
#     out.write(frame)

# cap.release()
# out.release()
    
   


In [17]:
import numpy as np
import cv2 as cv
 
cap = cv.VideoCapture("combscroll.mp4")
output_video_path = "C:/Users/supia/Documents/GitHub/Accessibility-AI-and-Video/modified_videos/testing_modified.mp4"
 
# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter(output_video_path, fourcc, 20.0, (640,  480))
 
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    frame = cv.flip(frame, 0)
 
    # write the flipped frame
    out.write(frame)
 
    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'):
        break
 
# Release everything if job is finished
cap.release()
out.release()
cv.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...


In [19]:
import numpy as np
import cv2 as cv
 
cap = cv.VideoCapture('combscroll.mp4')
 
while cap.isOpened():
    ret, frame = cap.read()
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
 
    cv.imshow('frame', gray)
    if cv.waitKey(1) == ord('q'):
        break
 
cap.release()
cv.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...


In [20]:
import numpy as np
import cv2 as cv
cap = cv.VideoCapture("combscroll.mp4")
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv.imshow('frame2', bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next
cv.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np

# Replace with your video file path
video_path = 'croppedvideo.mp4'

# Open video
cap = cv2.VideoCapture(video_path)

# Read the first frame
ret, frame1 = cap.read()
if not ret:
    print("Failed to read video.")
    cap.release()
    exit()

prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

frame_counter = 1

while True:
    # Read the next frame
    ret, frame2 = cap.read()
    if not ret:
        break

    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow between the two frames
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Normalize horizontal and vertical components
    # horz = cv2.normalize(flow[...,0], None, 0, 255, cv2.NORM_MINMAX)
    vert = cv2.normalize(flow[...,1], None, 0, 255, cv2.NORM_MINMAX)
    # horz = horz.astype('uint8')
    vert = vert.astype('uint8')

    # Show the components as images
    # cv2.imshow('Horizontal Component', horz)
    cv2.imshow('Vertical Component', vert)

    # Set the current frame as previous for the next iteration
    prvs = next.copy()

    # Wait for a key press
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        # cv2.imwrite(f'opticalflow_horz{frame_counter}.pgm', horz)
        cv2.imwrite(f'opticalflow_vert{frame_counter}.pgm', vert)

    frame_counter += 1

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np

# Replace with your video file path
video_path = 'your_video.mp4'

# Open video
cap = cv2.VideoCapture(video_path)

# Read the first frame
ret, frame1 = cap.read()
if not ret:
    print("Failed to read video.")
    cap.release()
    exit()

prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

frame_counter = 1

# Step size for drawing fewer flow lines for better readability
step = 16

while True:
    # Read the next frame
    ret, frame2 = cap.read()
    if not ret:
        break

    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Create a copy of the current frame for visualization
    vis = frame2.copy()

    # Draw flow vectors
    h, w = next.shape
    for y in range(0, h, step):
        for x in range(0, w, step):
            fx, fy = flow[y, x]
            end_point = (int(x + fx), int(y + fy))
            cv2.arrowedLine(vis, (x, y), end_point, color=(0, 255, 0), thickness=1, tipLength=0.4)

    # Display result
    cv2.imshow('Optical Flow Visualization', vis)

    # Prepare for next iteration
    prvs = next.copy()

    # Handle key input
    k = cv2.waitKey(30) & 0xff
    if k == 27:  # Esc to exit
        break
    elif k == ord('s'):
        cv2.imwrite(f'opticalflow_frame{frame_counter}.png', vis)

    frame_counter += 1

cap.release()
cv2.destroyAllWindows()


Failed to read video.


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


: 